# Test smple_weight, kernels, C, sklearn estimator

# Setup
Uncomment the next cell if STree is not already installed

In [1]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.datasets import make_classification, load_iris, load_wine
from sklearn.model_selection import train_test_split
from stree import Stree
import time

In [3]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [4]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

# data = load_creditcard(-5000) # Take all true samples + 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
data = load_creditcard(-1000) # Take all the samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]
# Set weights inverse to its count class in dataset
weights = np.ones(Xtrain.shape[0],) * 1.00244
weights[ytrain==1] = 1.99755
weights_test = np.ones(Xtest.shape[0],) * 1.00244
weights_test[ytest==1] = 1.99755 

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (1492, 28)  y.shape (1492,)
Fraud: 33.244% 496
Valid: 66.756% 996


# Tests

## Test smple_weights
Compute accuracy with weights in samples. The weights are set based on the inverse of the number of samples of each class

In [5]:
C = 23
print("Accuracy of Train without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtrain, ytrain))
print("Accuracy of Train with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtrain, ytrain))
print("Accuracy of Tests without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtest, ytest))
print("Accuracy of Tests with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtest, ytest))

Accuracy of Train without weights 0.9808429118773946
Accuracy of Train with    weights 0.9904214559386973
Accuracy of Tests without weights 0.9441964285714286
Accuracy of Tests with    weights 0.9375


## Test accuracy with different kernels
Compute accuracy on train and test set with default hyperparmeters of every kernel

In [6]:
random_state=1
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = Stree(C=7, kernel=kernel, random_state=random_state).fit(Xtrain, ytrain)
    accuracy_train = clf.score(Xtrain, ytrain)
    accuracy_test = clf.score(Xtest, ytest)
    time_spent = time.time() - now
    print(f"Time: {time_spent:.2f}s\tKernel: {kernel}\tAccuracy_train: {accuracy_train}\tAccuracy_test: {accuracy_test}")


Time: 0.13s	Kernel: linear	Accuracy_train: 0.9693486590038314	Accuracy_test: 0.9598214285714286
Time: 0.09s	Kernel: rbf	Accuracy_train: 0.9923371647509579	Accuracy_test: 0.953125
Time: 0.09s	Kernel: poly	Accuracy_train: 0.9913793103448276	Accuracy_test: 0.9375


## Test diferent values of C

In [7]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, kernel="linear", random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

************** C=0.001 ****************************
Classifier's accuracy (train): 0.9588
Classifier's accuracy (test) : 0.9487
root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4438
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0374
root - Down - Down, <cgaf> - Leaf class=1 belief= 0.984076 impurity=0.0313 counts=(array([0, 1]), array([  5, 309]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([1]))
root - Up, <cgaf> - Leaf class=0 belief= 0.947874 impurity=0.0988 counts=(array([0, 1]), array([691,  38]))

**************************************************
************** C=0.01 ****************************
Classifier's accuracy (train): 0.9588
Classifier's accuracy (test) : 0.9531
root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 

## Test iterator
Check different weays of using the iterator

In [8]:
#check iterator
for i in list(clf):
    print(i)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4438
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0367
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([315]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([6]))
root - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0846
root - Up - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([1]))
root - Up - Up, <cgaf> - Leaf class=0 belief= 0.957064 impurity=0.0822 counts=(array([0, 1]), array([691,  31]))


In [9]:
#check iterator again
for i in clf:
    print(i)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4438
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0367
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([315]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([6]))
root - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0846
root - Up - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([1]))
root - Up - Up, <cgaf> - Leaf class=0 belief= 0.957064 impurity=0.0822 counts=(array([0, 1]), array([691,  31]))


## Test STree is a sklearn estimator

In [10]:
# Make checks one by one
c = 0
checks = check_estimator(Stree(), generate_only=True)
for check in checks:
    c += 1
    print(c, check[1])
    check[1](check[0])

1 functools.partial(<function check_no_attributes_set_in_init at 0x12735b3b0>, 'Stree')
2 functools.partial(<function check_estimators_dtypes at 0x1273514d0>, 'Stree')
3 functools.partial(<function check_fit_score_takes_y at 0x1273513b0>, 'Stree')
4 functools.partial(<function check_sample_weights_pandas_series at 0x12734acb0>, 'Stree')
5 functools.partial(<function check_sample_weights_not_an_array at 0x12734add0>, 'Stree')
6 functools.partial(<function check_sample_weights_list at 0x12734aef0>, 'Stree')
7 functools.partial(<function check_sample_weights_shape at 0x12734d050>, 'Stree')
8 functools.partial(<function check_sample_weights_invariance at 0x12734d170>, 'Stree')
9 functools.partial(<function check_estimators_fit_returns_self at 0x1273564d0>, 'Stree')
10 functools.partial(<function check_estimators_fit_returns_self at 0x1273564d0>, 'Stree', readonly_memmap=True)
11 functools.partial(<function check_complex_data at 0x12734d320>, 'Stree')
12 functools.partial(<function check_dt

In [11]:
# Check if the classifier is a sklearn estimator
check_estimator(Stree())

## Compare to SVM

In [12]:
svc = SVC(C=7, kernel='rbf', gamma=.001, random_state=random_state)
clf = Stree(C=17, kernel='rbf', gamma=.001, random_state=random_state)
svc.fit(Xtrain, ytrain)
clf.fit(Xtrain, ytrain)
print("== Not Weighted ===")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
svc.fit(Xtrain, ytrain, weights)
clf.fit(Xtrain, ytrain, weights)
print("==== Weighted =====")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
print("*SVC test score ..:", svc.score(Xtest, ytest, weights_test))
print("*STree test score :", clf.score(Xtest, ytest, weights_test))

== Not Weighted ===
SVC train score ..: 0.9578544061302682
STree train score : 0.960727969348659
SVC test score ...: 0.9508928571428571
STree test score .: 0.9553571428571429
==== Weighted =====
SVC train score ..: 0.9636015325670498
STree train score : 0.9626436781609196
SVC test score ...: 0.9553571428571429
STree test score .: 0.9553571428571429
*SVC test score ..: 0.9447820728419238
*STree test score : 0.9447820728419238


In [13]:
print(clf)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4438
root - Down, <cgaf> - Leaf class=1 belief= 0.978261 impurity=0.0425 counts=(array([0, 1]), array([  7, 315]))
root - Up, <cgaf> - Leaf class=0 belief= 0.955679 impurity=0.0847 counts=(array([0, 1]), array([690,  32]))



## Test max_features

In [14]:
for max_features in [None, "auto", "log2", 7, .5, .1, .7]:
    now = time.time()
    print("*"*40)
    clf = Stree(random_state=random_state, max_features=max_features)
    clf.fit(Xtrain, ytrain)
    print(f"max_features {max_features} = {clf.max_features_}")
    print("Train score :", clf.score(Xtrain, ytrain))
    print("Test score .:", clf.score(Xtest, ytest))
    print(f"Took {time.time() - now:.2f} seconds")

****************************************
max_features None = 28
Train score : 0.9664750957854407
Test score .: 0.9642857142857143
Took 0.09 seconds
****************************************
max_features auto = 5
Train score : 0.9511494252873564
Test score .: 0.9441964285714286
Took 0.37 seconds
****************************************
max_features log2 = 4
Train score : 0.935823754789272
Test score .: 0.9330357142857143
Took 0.10 seconds
****************************************
max_features 7 = 7
Train score : 0.9568965517241379
Test score .: 0.9397321428571429
Took 3.36 seconds
****************************************
max_features 0.5 = 14
Train score : 0.960727969348659
Test score .: 0.9486607142857143
Took 112.42 seconds
****************************************
max_features 0.1 = 2
Train score : 0.8793103448275862
Test score .: 0.8839285714285714
Took 0.06 seconds
****************************************
max_features 0.7 = 19
Train score : 0.9655172413793104
Test score .: 0.955357142